In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().resolve().parent
RUNS_ROOT = PROJECT_ROOT / 'runs'
DATA_ROOT = PROJECT_ROOT / 'data' / 'MNIST'
print('Looking for runs in:', RUNS_ROOT)

In [ ]:
def load_all_runs(runs_root: Path) -> pd.DataFrame:
    runs_root = runs_root if runs_root.name == "runs" else (runs_root / "runs")
    rows = []
    if not runs_root.exists():
        print("No runs folder found:", runs_root)
        return pd.DataFrame()
    for exp_dir in sorted(runs_root.glob("*")):
        if not exp_dir.is_dir():
            continue
        for ts_dir in sorted(exp_dir.glob("*")):
            csv_path = ts_dir / "fl_log.csv"
            if csv_path.exists():
                try:
                    df = pd.read_csv(csv_path)
                    df["exp"] = exp_dir.name
                    df["ts"] = ts_dir.name
                    rows.append(df)
                except Exception as e:
                    print("Failed to read", csv_path, e)
    if not rows:
        return pd.DataFrame()
    return pd.concat(rows, ignore_index=True)

df = load_all_runs(RUNS_ROOT)
print("Loaded rows:", len(df))
df.head(3)


In [ ]:
# Federated evaluation across rounds & nodes using base + delta reconstruction.
# - Auto-discovers files like:
#     models/global_round_{r}_base.pt (or fallback: global_round_{r}.pt)
#     updates/round_{r}_node_{i}_delta.pt
# - Reconstructs each node's local model: base + delta
# - Evaluates accuracy on a test loader (replace with your real test loader)
# - Summarizes results in a DataFrame and saves CSV
#
# If no artifacts are found at the default path, this will run a synthetic demo
# so you can see the full pipeline and results format.
#
# === HOW TO ADAPT ===
# 1) Set RUN_DIR to your run folder containing "models" and "updates".
# 2) Replace MODEL_FACTORY() with your actual model constructor.
# 3) Replace build_test_loader() with your real test DataLoader.
# 4) If your .pt files are full models (not state_dict), adjust load logic.
#
# Files will be saved to /mnt/data, including results CSV.
import os
import re
from pathlib import Path
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from typing import Dict, Any
# -----------------------------
# Config: paths
# -----------------------------
# Try to auto-detect a likely run dir under /mnt/data/runs/benign/*
CANDIDATE = None
runs_root = RUNS_ROOT / "benign"
if runs_root.exists():
    # pick the most recently modified child that has both models/ and updates/
    candidates = []
    for p in runs_root.glob("*"):
        if p.is_dir() and (p / "models").exists() and (p / "updates").exists():
            candidates.append((p.stat().st_mtime, p))
    if candidates:
        CANDIDATE = sorted(candidates, key=lambda x: x[0], reverse=True)[0][1]
# Allow the user to change this path if needed:
RUN_DIR = CANDIDATE if CANDIDATE else (runs_root / "20250813-224933")
MODELS_DIR = RUN_DIR / "models"
UPDATES_DIR = RUN_DIR / "updates"
# -----------------------------
# Config: model + test loader
# -----------------------------
NUM_CLASSES = 10
IMAGE_SHAPE = (1, 28, 28)
from flsim.models import LinearMNIST
from torchvision import datasets, transforms
def MODEL_FACTORY():
    return LinearMNIST()
def build_test_loader():
    transform = transforms.ToTensor()
    dataset = datasets.MNIST(root=DATA_ROOT, train=False, download=False, transform=transform)
    subset = torch.utils.data.Subset(dataset, range(1000))
    return DataLoader(subset, batch_size=64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -----------------------------
# Helpers
# -----------------------------
def reconstruct_full_state(base_state: Dict[str, torch.Tensor],
                           delta_state: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    full = {}
    for k, base_tensor in base_state.items():
        if k in delta_state:
            full[k] = base_tensor + delta_state[k]
        else:
            full[k] = base_tensor
    for k, v in delta_state.items():
        if k not in full:
            full[k] = v
    return full
@torch.no_grad()
def evaluate(model: nn.Module, loader: DataLoader) -> Dict[str, Any]:
    model.eval()
    total, correct, total_loss = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.numel()
        total_loss += loss.item() * yb.size(0)
    acc = correct / total if total > 0 else 0.0
    avg_loss = total_loss / total if total > 0 else 0.0
    return {"acc": acc, "loss": avg_loss, "n": total}
def find_rounds_nodes(updates_dir: Path):
    # pattern: round_{r}_node_{i}_delta.pt
    pat = re.compile(r"round_(\d+)_node_(\d+)_delta\.pt$")
    mapping = {}
    if not updates_dir.exists():
        return mapping
    for f in updates_dir.glob("*.pt"):
        m = pat.search(f.name)
        if m:
            r = int(m.group(1))
            i = int(m.group(2))
            mapping.setdefault(r, []).append((i, f))
    # sort nodes by id per round
    for r in mapping:
        mapping[r] = sorted(mapping[r], key=lambda x: x[0])
    return dict(sorted(mapping.items(), key=lambda x: x[0]))
def load_base_for_round(models_dir: Path, r: int):
    # prefer global_round_{r}_base.pt else fallback to global_round_{r}.pt
    base1 = models_dir / f"global_round_{r-1}_base.pt"
    base2 = models_dir / f"global_round_{r-1}.pt"
    if base1.exists():
        return torch.load(base1, map_location="cpu")
    if base2.exists():
        return torch.load(base2, map_location="cpu")
    return None
# -----------------------------
# Main evaluation
# -----------------------------
test_loader = build_test_loader()
results = []
artifact_found = MODELS_DIR.exists() and UPDATES_DIR.exists()
if artifact_found:
    rounds = find_rounds_nodes(UPDATES_DIR)
    if not rounds:
        print(f"[Info] No update deltas found under {UPDATES_DIR}. Running synthetic demo instead.")
        artifact_found = False
if artifact_found:
    print(f"[Using artifacts from] {RUN_DIR}")
    for r, node_list in rounds.items():
        base_state = load_base_for_round(MODELS_DIR, r)
        if base_state is None:
            print(f"[Warn] No base model found for round {r} in {MODELS_DIR}, skipping this round.")
            continue
        for node_id, delta_path in node_list:
            try:
                delta_state = torch.load(delta_path, map_location="cpu")
            except Exception as e:
                print(f"[Error] Could not load delta {delta_path}: {e}")
                continue
            # reconstruct
            reconstructed_state = reconstruct_full_state(base_state, delta_state)
            model = MODEL_FACTORY().to(device)
            try:
                missing, unexpected = model.load_state_dict(reconstructed_state, strict=False)
            except Exception as e:
                print(f"[Error] load_state_dict failed for round {r}, node {node_id}: {e}")
                continue
            # evaluate
            metrics = evaluate(model, test_loader)
            results.append({
                "round": r,
                "node": node_id,
                "acc": metrics["acc"],
                "loss": metrics["loss"],
                "n": metrics["n"],
                "missing_keys": len(missing),
                "unexpected_keys": len(unexpected),
                "delta_file": str(delta_path.name)
            })
else:
    # Synthetic demo so the pipeline is visible and results are produced
    print("[Demo] No real artifacts detected. Running a synthetic demonstration.")
    # Create a base model
    base_model = MODEL_FACTORY().to(device)
    base_state = base_model.state_dict()
    # Simulate 2 rounds × 3 nodes with small random deltas
    for r in range(2):
        for node_id in range(3):
            delta_state = {k: torch.randn_like(v) * 0.01 for k, v in base_state.items()}
            reconstructed_state = reconstruct_full_state(base_state, delta_state)
            model = MODEL_FACTORY().to(device)
            model.load_state_dict(reconstructed_state, strict=False)
            metrics = evaluate(model, test_loader)
            results.append({
                "round": r,
                "node": node_id,
                "acc": metrics["acc"],
                "loss": metrics["loss"],
                "n": metrics["n"],
                "missing_keys": 0,
                "unexpected_keys": 0,
                "delta_file": f"synthetic_round_{r}_node_{node_id}_delta"
            })
# -----------------------------
# Save & show results
# -----------------------------
df = pd.DataFrame(results).sort_values(["round", "node"]).reset_index(drop=True)
print(df)
# out_csv = "/mnt/data/per_node_accuracy_by_round.csv"
# df.to_csv(out_csv, index=False)
# Display table to the user in the UI
# %pip install caas_jupyter_tools
# from caas_jupyter_tools import display_dataframe_to_user
# display_dataframe_to_user("Per-node accuracy by round", df)
# print(f"\nSaved results to: {out_csv}")
# print(f"RUN_DIR used: {RUN_DIR}")
